In [14]:
import pandas as pd
import numpy as np


In [15]:
hotellook = pd.read_csv('hotellook_cleaned.csv')
opentrip = pd.read_csv('opentripmap_cleaned.csv')
otello = pd.read_csv('otello_cleaned.csv')

In [13]:
hotellook['source'] = 'hotellook'
otello['source'] = 'otello'

In [16]:
# Приводим названия к нижнему регистру и убираем лишние пробелы
hotellook['name_clean'] = hotellook['name'].str.lower().str.strip()
otello['name_clean'] = otello['name'].str.lower().str.strip()

In [17]:
# Округляем координаты до 2 знаков после запятой (это примерно 1 км, можно подобрать по ситуации)
hotellook['lat_round'] = hotellook['latitude'].round(2)
hotellook['lon_round'] = hotellook['longitude'].round(2)
otello['lat_round'] = otello['latitude'].round(2)
otello['lon_round'] = otello['longitude'].round(2)

# Создаем ключ для группировки по округленным координатам
hotellook['geo_key'] = hotellook['lat_round'].astype(str) + "_" + hotellook['lon_round'].astype(str)
otello['geo_key'] = otello['lat_round'].astype(str) + "_" + otello['lon_round'].astype(str)

In [19]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.2 MB/s eta 0:00:00


In [20]:
from rapidfuzz import fuzz

In [33]:
df_all = pd.concat([hotellook, otello], ignore_index=True)
print("Размер объединенного датасета до дедупликации:", df_all.shape)

# Удаление дубликатов
# Для каждой группы по geo_key будем сравнивать названия и считать дубликатом, если схожесть >= threshold.
threshold = 85
indices_to_drop = set()

for key, group in df_all.groupby('geo_key'):
    # Если в группе больше одной записи, начинается процесс поиска дубликатов
    if len(group) > 1:
        seen_indices = []
        # Проходим по индексам группы
        for idx in group.index:
            current_name = df_all.at[idx, 'name_clean']
            duplicate_found = False
            # Сравниваем с уже просмотренными записями в данной группе
            for seen_idx in seen_indices:
                seen_name = df_all.at[seen_idx, 'name_clean']
                # Вычисляем сходство между current_name и seen_name с помощью fuzz.token_set_ratio (токенизация названия + рассчет расстояния Левенштейна)
                score = fuzz.token_set_ratio(current_name, seen_name)
                if score >= threshold:
                    duplicate_found = True
                    break
            if duplicate_found:
                indices_to_drop.add(idx)
            else:
                seen_indices.append(idx)

Размер объединенного датасета до дедупликации: (52181, 37)


In [34]:
df_merged = df_all.drop(index=list(indices_to_drop)).reset_index(drop=True)
print("Размер объединенного датасета после дедупликации:", df_merged.shape)

df_merged.to_csv('merged_hotels.csv', index=False)

Размер объединенного датасета после дедупликации: (23218, 37)


In [35]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23218 entries, 0 to 23217
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  21001 non-null  float64
 1   name                19217 non-null  object 
 2   address             20922 non-null  object 
 3   location_id         21001 non-null  object 
 4   latitude            23218 non-null  float64
 5   longitude           23218 non-null  float64
 6   stars               21001 non-null  float64
 7   rating              21001 non-null  float64
 8   reviews_count       21001 non-null  float64
 9   min_price           21001 non-null  float64
 10  distance_to_center  21001 non-null  float64
 11  property_type       21001 non-null  object 
 12  photos_count        21001 non-null  float64
 13  amenities           9602 non-null   object 
 14  badges              1350 non-null   object 
 15  check_in            0 non-null      float64
 16  chec

In [37]:
df_merged.columns

Index(['id', 'name', 'address', 'location_id', 'latitude', 'longitude',
       'stars', 'rating', 'reviews_count', 'min_price', 'distance_to_center',
       'property_type', 'photos_count', 'amenities', 'badges', 'check_in',
       'check_out', 'popularity', 'popularity2', 'trending_speed',
       'pois_distances', 'weights', 'district_name', 'name_clean', 'lat_round',
       'lon_round', 'geo_key', 'checkinTime', 'checkoutTime', 'amenityFeature',
       'address_Locality', 'street_Address', 'Otello_starRating',
       'Otello_ratingValue', 'Otello_ratingCount', 'Otello_bestRating',
       'Otello_worstRating'],
      dtype='object')